In [1]:
import sys;
from esinet.evaluate import eval_mse
from esinet import Simulation,Net, util, evaluate
from esinet.util import calculate_source
from forward import create_forward_model, get_info
from matplotlib import pyplot as plt
import numpy as np
import mne

In [4]:
# Create generic Forward Model 设置频率
info = get_info(sfreq=256)
# 生成前向模型，ico2表示324个源，ico3表示1284个源，oct5表示2052个源,其间距为10mm ico4表示5124个源,
fwd = create_forward_model(info=info, sampling='ico3')
leadfield, pos = util.unpack_fwd(fwd)[1:3]
print(fwd)
print(leadfield)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.1

<Forward | MEG channels: 0 | EEG channels: 60 | Source space: Surface with 1284 vertices | Source orientation: Fixed>
[[-46.906826  -40.41705   -16.183111  ...  24.814037  -34.563293
   22.82753  ]
 [-52.740463  -41.824825   -3.9244251 ...  40.904682  -39.802998
   36.42079  ]
 [-53.084373  -41.05366    11.623575  ...  61.38438   -38.46862
   53.22305  ]
 ...
 [ 36.584454   77.79619    15.44015   ... -45.12664     7.7374344
  -50.830994 ]
 [ 23.741581   75.113      26.675865  ... -46.488968   10.753365
  -50.925224 ]
 [  7.747348   41.80889    38.380733  ... -48.96788    18.615059
  -51.181976 ]]


[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.1s finished


In [5]:
mm=np.array(leadfield)
print(leadfield.shape)

(60, 1284)


In [ ]:
settings = {
    'method': 'standard', # 模拟源信号时间的方式
    'number_of_sources': (1, 2), # 源的数量范围
    'extents':  (5, 40),  # 源的直径范围 单位是mm
    'amplitudes': (3, 10), # 源的单流大小范围，单位是nAm，前面的n表示纳，是10的-9次方，1 nAm = 0.001 µAm，默认(0.001, 100)
    'shapes': 'mixed', # 源的形状，可以是高斯分布或者均匀分布，以及二者混合分布
    'duration_of_trial': 0.1, #试次的时间长度，以秒为单位
    'sample_frequency': 256, # 数据的采样频率
    'target_snr': 10, # 目标信噪比范围，默认（2，20）
    'beta': (0, 3), 
    'source_spread': "region_growing", # 源的空间分布
    'source_number_weighting': True, # 指示是否对源的数量进行加权，也就是从设定的范围内随机选择一个值作为源的数量
    'source_time_course': "random", # 源的时间过程类型是随机
}
sim = Simulation(fwd, info, settings=settings, verbose=True)
sim.simulate(n_samples=20000)

Simulate Source
Simulating data based on sparse patches.


100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [01:15<00:00, 263.19it/s]


Converting Source Data to mne.SourceEstimate object


100%|██████████████████████████████████████████████████████████████████████████| 20000/20000 [00:02<00:00, 7081.58it/s]


source data shape:  (1284, 26) (1284, 26)

Project sources to EEG...
<class 'list'> (60, 26)

Create EEG trials with noise...


100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [06:07<00:00, 54.39it/s]



Convert EEG matrices to a single instance of mne.Epochs...
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activ

In [ ]:
# 提取
def prep_data(sim):
    
    x = np.squeeze(np.stack([eeg.average().data.astype(np.float32) for eeg in sim.eeg_data]))
    y = np.squeeze(np.stack([src.data.astype(np.float32) for src in sim.source_data]))
    
    for i, (x_sample, y_sample) in enumerate(zip(x, y)):
        # X[i] = np.stack([(x - np.mean(x)) / np.std(x) for x in x_sample.T], axis=0).T
        # y[i] = np.stack([ y / np.max(abs(y)) for y in y_sample.T], axis=0).T

        x[i] = np.stack([x - np.mean(x) for x in x_sample.T], axis=0).T
        x[i] /= x[i].std()
        y[i] /= np.max(abs(y[i]))
        
    x = np.swapaxes(x, 1,2)
    y = np.swapaxes(y, 1,2)
    
    return x, y

In [ ]:
y = np.squeeze(np.stack([src.data.astype(np.float32) for src in sim.source_data]))
print(y.shape)

In [ ]:
# 提取源信号
sources = sim.source_data
sources = [source.data for source in sources]
print(type(sources))
print(type(sources[0]))

In [ ]:
y=np.array(sources)
print(type(y))
print(y.shape)

In [ ]:
x=np.array(sim.eeg_data)
print(type(x))
print(x.shape)
x=x.squeeze()
print(x.shape)

In [ ]:
np.save('x.npy', x)

In [ ]:
np.save('y.npy', y)

In [ ]:
arr=y[20,22,:]
list_version = arr.tolist()
print(list_version)

def custom_prep_source(data):
    
    # 将数据类型转换为 np.float32
    data = data.astype(np.float32)
    
    for i, y_sample in enumerate(data):
        data[i] /= np.max(abs(data[i]))

    data = np.swapaxes(data, 1,2)
    print("The shape of source is", data.shape)
    
    return data

arr2=custom_prep_source(y)
arr3=arr2[20,22,:]
list_version2 = arr3.tolist()
print(list_version2)

In [ ]:
x= np.load('D:/jupyter_note/SWX_source/曹薇/X.npy')
y= np.load('D:/jupyter_note/SWX_source/曹薇/y.npy')
arr=x[5,13,:]
list_version = arr.tolist()
print(list_version)

In [ ]:
x_sample = np.array([[1, 2],
                     [3, 4],
                     [5, 6]])

# Normalize each column by subtracting its mean
x_normalized = np.stack([col - np.mean(col) for col in x_sample.T], axis=0).T

print(x_normalized)